In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train = pd.read_csv('Titanic_train.csv')
test=pd.read_csv('Titanic_test.csv')

In [3]:
av_age = train.groupby('Pclass')
av_age.mean()

,PassengerId,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,,
1,461.597222,0.629630,38.233441,0.416667,0.356481,84.154687
2,445.956522,0.472826,29.877630,0.402174,0.380435,20.662183
3,439.154786,0.242363,25.140620,0.615071,0.393075,13.675550


In [4]:
def impute_age_train(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1:
            return 38.23
        elif Pclass == 2:
            return 29.87
        else:
            return 25.14
        
    else:
        return Age

In [5]:
train['Age'] = train[['Age','Pclass']].apply(impute_age_train,axis=1)
train=train.dropna(subset = ['Embarked'])
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [6]:
av_age = test.groupby('Pclass')
av_age.mean()

,PassengerId,Age,SibSp,Parch,Fare
Pclass,,,,,
1,1098.224299,40.918367,0.476636,0.383178,94.280297
2,1117.935484,28.777500,0.376344,0.344086,22.202104
3,1094.178899,24.027945,0.463303,0.417431,12.459678


In [7]:
def impute_age_test(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        
        if Pclass == 1:
            return 40.9
        elif Pclass == 2:
            return 28.77
        else:
            return 24.02
        
    else:
        return Age

In [8]:
test['Age'] = test[['Age','Pclass']].apply(impute_age_test,axis=1)
missing_val_count_by_column = (test.isnull().sum())
print(missing_val_count_by_column)

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [9]:
X_train = train.drop(['PassengerId', 'Survived', 'Fare', 'Name','Ticket', 'Fare', 'Cabin'], axis=1)
y_train = train.iloc[:, 1]
X_test = test.drop(['PassengerId', 'Fare', 'Name','Ticket', 'Fare','Cabin'], axis=1)

In [10]:
X_train=pd.get_dummies(X_train, prefix=['Sex', 'Embarked'], columns=['Sex', 'Embarked'])

In [11]:
X_test=pd.get_dummies(X_test, prefix=['Sex', 'Embarked'], columns=['Sex', 'Embarked'])

In [12]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:    6.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    6.7s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [25, 50, 75], 'learning_rate': [0.01, 0.05, 0.1], 'num_leaves': [300, 900, 1200], 'n_estimators': [200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [13]:
y_pred = grid_search.predict(X_test)

In [14]:
PassengerId = np.array(test["PassengerId"]).astype(int)
my_submission_rfc = pd.DataFrame(y_pred, PassengerId, columns = ["Survived"])
my_submission_rfc.to_csv("submission_titanicLGBM.csv", index_label = ["PassengerId"])

In [15]:
f=pd.read_csv('submission_titanicLGBM.csv')

In [16]:
f.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
